<a href="https://colab.research.google.com/github/anaisidoro/ASOCIO-2025---POwer-Girls/blob/main/Instance2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
!pip install pulp
from pulp import *
import time
# --- INÍCIO DA MEDIÇÃO ---
start_time = time.time()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 69.2 MB/s eta 0:00:00


In [ ]:
#Input data
with open('/content/instance2.json') as f:
    instance = json.load(f)

#Sets and parameters
days = ["L", "Ma", "Mi", "J", "V"]
employee = instance['Employees']
desks = instance['Desks']
zones = instance['Zones']
desks_per_zone = instance['Desks_Z']
desks_per_employee = instance['Desks_E']
employee_per_group = instance['Employees_G']
preferable_days = instance['Days_E']
weight = 5
groups = [g for g in employee_per_group]

#Update weight matrix
weights = [[1 for _ in employee] for _ in days]
for e in employee:
    for i in range(len(preferable_days[e])):
        day_index = days.index(preferable_days[e][i])
        weights[day_index][employee.index(e)]=weight

#Identify the employees that can use each desk
employee_per_desk = {}
for desk_id in desks:
    employees_in_desk = []
    for e in employee:
        possible_desks = desks_per_employee[e]
        for m in possible_desks:
            if m == desk_id:
                employees_in_desk.append(e)
    employee_per_desk[desk_id]=employees_in_desk

In [ ]:
#Problem Initialization
assignment = LpProblem("StudentChallenge", LpMaximize)

#Decision Variables
x = LpVariable.dicts("x", [(d,e,k) for d in days for e in employee for k in desks if e in employee_per_desk[k]], cat = "Binary")
u = LpVariable.dicts("u", [(d,g) for d in days for g in groups], cat = "Binary")

#Objective Function
assignment += lpSum([weights[days.index(d)][employee.index(e)]*x[d,e, k] for d in days for e in employee for k in desks if e in employee_per_desk[k]])

#Constraint 1 = Each employee must stay at the office at least twice a week
for e in employee:
    assignment += lpSum([x[d,e,k] for d in days for k in desks if e in employee_per_desk[k]]) >= 2

#Constraint 2 = Each desk must be occupied by at most one specific employees at each day
for k in desks:
    for d in days:
        assignment += lpSum([x[d,e,k] for e in employee_per_desk[k]]) <= 1

#Constraint 3 - Each employee must be assigned at most once a day
for d in days:
    for e in employee:
        assignment += lpSum([x[d,e,k] for k in desks if e in employee_per_desk[k]]) <= 1

#Constraint 4 - Each desk must be assigned at most once a day
for k in desks:
    for d in days:
        assignment += lpSum([x[d,e,k] for e in employee if e in employee_per_desk[k]]) <= 1


#Constraint 5 = Employees should not be isolated in a zone
for d in days:
    for z in zones:
        for g in groups:
            for m in employee_per_group[g]:
                assignment += lpSum([x[d,m,k] for k in desks_per_employee[m] if k in desks_per_zone[z]]) <= lpSum([x[d,e,k] for e in employee_per_group[g] if e!=m for k in desks_per_employee[e] if k in desks_per_zone[z]])


#Constraint 6 = Each group should have one meeting per week
for g in groups:
    assignment += lpSum([u[d,g] for d in days]) >= 1

#Constraint 7 = All the members of a group should be present once a week
for d in days:
    for g in groups:
        employees_group = employee_per_group[g]
        assignment += lpSum([x[d,e,k] for e in employees_group for k in desks if e in employee_per_desk[k]]) >= len(employees_group)*u[d,g]

In [ ]:
#Solving the model
solve_the_model = assignment.solve()
print(f"Status do problema: {LpStatus[solve_the_model]}")

#Print variables
for var in assignment.variables():
    if var.varValue > 0:
        print(f"{var.name}={var.varValue}")

#Print objective function value
print(f"Total Happiness: {value(assignment.objective)}")

Status do problema: Optimal
u_('J',_'G2')=1.0
u_('L',_'G1')=1.0
u_('Ma',_'G3')=1.0
u_('Mi',_'G0')=1.0
x_('J',_'E0',_'D6')=1.0
x_('J',_'E10',_'D2')=1.0
x_('J',_'E11',_'D5')=1.0
x_('J',_'E12',_'D0')=1.0
x_('J',_'E13',_'D3')=1.0
x_('J',_'E14',_'D7')=1.0
x_('J',_'E15',_'D4')=1.0
x_('J',_'E16',_'D1')=1.0
x_('J',_'E3',_'D8')=1.0
x_('L',_'E11',_'D3')=1.0
x_('L',_'E12',_'D2')=1.0
x_('L',_'E16',_'D7')=1.0
x_('L',_'E17',_'D8')=1.0
x_('L',_'E5',_'D5')=1.0
x_('L',_'E6',_'D6')=1.0
x_('L',_'E7',_'D1')=1.0
x_('L',_'E8',_'D0')=1.0
x_('L',_'E9',_'D4')=1.0
x_('Ma',_'E15',_'D6')=1.0
x_('Ma',_'E16',_'D1')=1.0
x_('Ma',_'E17',_'D4')=1.0
x_('Ma',_'E18',_'D0')=1.0
x_('Ma',_'E19',_'D8')=1.0
x_('Ma',_'E2',_'D7')=1.0
x_('Ma',_'E4',_'D5')=1.0
x_('Ma',_'E6',_'D2')=1.0
x_('Ma',_'E9',_'D3')=1.0
x_('Mi',_'E0',_'D6')=1.0
x_('Mi',_'E1',_'D4')=1.0
x_('Mi',_'E10',_'D2')=1.0
x_('Mi',_'E14',_'D1')=1.0
x_('Mi',_'E2',_'D0')=1.0
x_('Mi',_'E3',_'D3')=1.0
x_('Mi',_'E4',_'D8')=1.0
x_('Mi',_'E5',_'D5')=1.0
x_('Mi',_'E8',_'D7')=1.

In [ ]:
#Print results organized per day
for d in days:
    print(f"-------------------Day: {d}----------------------")
    for k in desks:
        for e in employee:
            if e in employee_per_desk[k]:
                if x[d, e, k].value() == 1:
                    print(f"Desk: {k}, employee:{e}")
    for g in groups:
        if u[d, g].value() == 1:
            group_employees = employee_per_group[g]
            print(f"Meeting of the day: Group {g} - {group_employees}")

-------------------Day: L----------------------
Desk: D0, employee:E8
Desk: D1, employee:E7
Desk: D2, employee:E12
Desk: D3, employee:E11
Desk: D4, employee:E9
Desk: D5, employee:E5
Desk: D6, employee:E6
Desk: D7, employee:E16
Desk: D8, employee:E17
Meeting of the day: Group G1 - ['E5', 'E6', 'E7', 'E8', 'E9']
-------------------Day: Ma----------------------
Desk: D0, employee:E18
Desk: D1, employee:E16
Desk: D2, employee:E6
Desk: D3, employee:E9
Desk: D4, employee:E17
Desk: D5, employee:E4
Desk: D6, employee:E15
Desk: D7, employee:E2
Desk: D8, employee:E19
Meeting of the day: Group G3 - ['E15', 'E16', 'E17', 'E18', 'E19']
-------------------Day: Mi----------------------
Desk: D0, employee:E2
Desk: D1, employee:E14
Desk: D2, employee:E10
Desk: D3, employee:E3
Desk: D4, employee:E1
Desk: D5, employee:E5
Desk: D6, employee:E0
Desk: D7, employee:E8
Desk: D8, employee:E4
Meeting of the day: Group G0 - ['E0', 'E1', 'E2', 'E3', 'E4']
-------------------Day: J----------------------
Desk: D0, 

In [ ]:
# --- FIM DA MEDIÇÃO ---
end_time = time.time()

# Calcula o tempo total decorrido
time_taken = end_time - start_time

# Imprime o tempo de execução formatado
print(f"\n--- Tempo de Execução do Código ---")
print(f"O processo completo levou {time_taken:.4f} segundos para ser executado.")
print(f"-----------------------------------")


--- Tempo de Execução do Código ---
O processo completo levou 20.2129 segundos para ser executado.
-----------------------------------
